In [50]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split

In [52]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [53]:
x_train = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/HIKARI (Prem)/Datasets/CASE II/V3 - Entire Labels/Hikari_train_2labels.pkl')
y_train = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/HIKARI (Prem)/Datasets/CASE II/V3 - Entire Labels/Hikari_train_2labels_y.pkl')

x_val = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/HIKARI (Prem)/Datasets/CASE II/V3 - Entire Labels/Hikari_val_2labels.pkl')
y_val = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/HIKARI (Prem)/Datasets/CASE II/V3 - Entire Labels/Hikari_val_2labels_y.pkl')

x_test = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/HIKARI (Prem)/Datasets/CASE II/V3 - Entire Labels/Hikari_test_2labels.pkl')
y_test = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/HIKARI (Prem)/Datasets/CASE II/V3 - Entire Labels/Hikari_test_2labels_y.pkl')

In [54]:
x_train = x_train.drop(['is_Attack', 'is_Normal'], axis=1)

In [55]:
x_val = x_val.drop(['is_Attack', 'is_Normal'], axis=1)

In [56]:
x_test = x_test.drop(['is_Attack', 'is_Normal'], axis=1)

In [57]:
# x_test

In [58]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)




In [59]:
class Data(Dataset):
  def __init__(self, x, y, scale_data = True):
      self.x = torch.FloatTensor(x)
      self.y = torch.LongTensor(y)
      self.len = self.y.shape[0]

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.len

In [60]:
train = Data(x_train, y_train)
val = Data(x_val, y_val)
test = Data(x_test, y_test)

train_loader = DataLoader(train, batch_size = 1024, shuffle = True)
val_loader = DataLoader(val, batch_size = len(val), shuffle = True)
test_loader = DataLoader(test, batch_size = len(test), shuffle = True)

In [61]:
linear1 = torch.nn.Linear(79, 256, bias=False)
linear2 = torch.nn.Linear(256, 128, bias=False)
linear3 = torch.nn.Linear(128, 64, bias=False)
linear4 = torch.nn.Linear(64, 2, bias=False)
relu = torch.nn.ReLU()

In [62]:
class Autoencoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = nn.Sequential(
        nn.Linear(79, 256, bias=False),
        nn.ReLU(),
        nn.Linear(256, 128, bias=False),
        nn.ReLU(),
        nn.Linear(128, 64, bias=False),
        nn.ReLU(),
        nn.Linear(64, 1, bias = False)
    )
    self.decoder = nn.Sequential(
        nn.Linear(64, 128, bias=False),
        nn.ReLU(),
        nn.Linear(128, 256, bias=False),
        nn.ReLU(),
        nn.Linear(256, 79, bias=False)
    )

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded




In [63]:
model = Autoencoder().to(device)
print(model)



Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=79, out_features=256, bias=False)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=False)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=False)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=False)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=False)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=79, bias=False)
  )
)


In [64]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [65]:
# total_batch = len(train_loader)
model.train()

for epoch in range(20):

  for x_train, y_train in train_loader:
    x_train = x_train.to(device)
    y_train = y_train.to(device)

    optimizer.zero_grad()

    pred = model(x_train).to(device)
    cost = loss(pred, y_train).to(device)
    correct_prediction = torch.argmax(pred, 1) == y_train
    accuracy = correct_prediction.float().mean()

    cost.backward()
    optimizer.step()

    # avg_cost += cost / total_batch

  print("Epoch {:4d} / {} Cost: {:.2f} Training Accuracy {:.2f}%".
          format(epoch+1, 20, cost.item(), accuracy*100))

Epoch    1 / 20 Cost: 1.23 Training Accuracy 57.81%
Epoch    2 / 20 Cost: 0.53 Training Accuracy 78.12%
Epoch    3 / 20 Cost: 0.47 Training Accuracy 78.12%
Epoch    4 / 20 Cost: 0.34 Training Accuracy 87.50%
Epoch    5 / 20 Cost: 0.33 Training Accuracy 82.81%
Epoch    6 / 20 Cost: 0.24 Training Accuracy 90.62%
Epoch    7 / 20 Cost: 0.21 Training Accuracy 92.19%
Epoch    8 / 20 Cost: 0.22 Training Accuracy 89.06%
Epoch    9 / 20 Cost: 0.20 Training Accuracy 92.19%
Epoch   10 / 20 Cost: 0.19 Training Accuracy 93.75%
Epoch   11 / 20 Cost: 0.20 Training Accuracy 87.50%
Epoch   12 / 20 Cost: 0.22 Training Accuracy 87.50%
Epoch   13 / 20 Cost: 0.15 Training Accuracy 95.31%
Epoch   14 / 20 Cost: 0.16 Training Accuracy 95.31%
Epoch   15 / 20 Cost: 0.21 Training Accuracy 95.31%
Epoch   16 / 20 Cost: 0.24 Training Accuracy 85.94%
Epoch   17 / 20 Cost: 0.13 Training Accuracy 93.75%
Epoch   18 / 20 Cost: 0.17 Training Accuracy 95.31%
Epoch   19 / 20 Cost: 0.21 Training Accuracy 89.06%
Epoch   20 /

In [66]:
model.eval()
with torch.no_grad():
  for x_val, y_val in val_loader:
    x_val = x_val.to(device)
    y_val = y_val.to(device)
    prediction = model(x_val).to(device)

    correct_prediction = torch.argmax(prediction, 1) == y_val
    accuracy = correct_prediction.float().mean()

  print('Validation Accuracy: {:.2f}%'.format(accuracy.item()*100))

Validation Accuracy: 92.25%


In [67]:
model.eval()
with torch.no_grad():
  for x_test, y_test in test_loader:
    x_test = x_test.to(device)
    y_test = y_test.to(device)
    prediction = model(x_test).to(device)

    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()

  print('Testing Accuracy: {:.2f}%'.format(accuracy.item()*100))

Testing Accuracy: 92.71%


In [68]:
#Bulid confusion matrix using the f1 score

from sklearn.metrics import classification_report, confusion_matrix

y_pred = model(x_test).to(device)
y_pred = torch.argmax(y_pred, 1).to(device)

target_names = ['is_Normal', 'is_Attack']

print(classification_report(y_test.cpu(), y_pred.cpu(), target_names=target_names, digits=4))
print(confusion_matrix(y_test.cpu(), y_pred.cpu()))

              precision    recall  f1-score   support

   is_Normal     0.9968    0.8588    0.9226      4000
   is_Attack     0.8731    0.9972    0.9311      3900

    accuracy                         0.9271      7900
   macro avg     0.9350    0.9280    0.9268      7900
weighted avg     0.9358    0.9271    0.9268      7900

[[3435  565]
 [  11 3889]]
